In [3]:
test_input = '''class: 1-3 or 5-7
row: 6-11 or 33-44
seat: 13-40 or 45-50

your ticket:
7,1,14

nearby tickets:
7,3,47
40,4,50
55,2,20
38,6,12'''.split('\n\n')

In [67]:
test_input_2 = '''class: 0-1 or 4-19
row: 0-5 or 8-19
seat: 0-13 or 16-19

your ticket:
11,12,13

nearby tickets:
3,9,18
15,1,5
5,14,9
'''.split('\n\n')

In [4]:
test_input

['class: 1-3 or 5-7\nrow: 6-11 or 33-44\nseat: 13-40 or 45-50',
 'your ticket:\n7,1,14',
 'nearby tickets:\n7,3,47\n40,4,50\n55,2,20\n38,6,12']

In [57]:
def prep_input(raw_input):
    items, mine, others = raw_input
    intervals = {}
    for item_type in items.split('\n'):
        name = item_type.split(':')[0]
        first_interv = item_type.split(' ')[-3]
        deb1, fin1 = first_interv.split('-')
        second_interv = item_type.split(' ')[-1]
        deb2, fin2 = second_interv.split('-')
        intervals[name] = (int(deb1), int(fin1), int(deb2), int(fin2))
        #print(f'{name}: {intervals[name]}')

    my_ticket = mine.split('\n')[1]
    my_ticket = [int(n) for n in my_ticket.split(',')]
    #print(f'my ticket: {my_ticket}')

    other_tickets = []
    split_others = filter(None, others.split('\n')[1:])
    for other in split_others:
        other_tickets.append([int(n) for n in other.split(',')])

    #print(f'other tickets: {other_tickets}')
    return intervals, my_ticket, other_tickets

In [68]:
prep_input(test_input_2)

({'class': (0, 1, 4, 19), 'row': (0, 5, 8, 19), 'seat': (0, 13, 16, 19)},
 [11, 12, 13],
 [[3, 9, 18], [15, 1, 5], [5, 14, 9]])

In [142]:
def nb_invalid(intervals, my_ticket, other_tickets):
    total_invalid = 0
    valid_tickets = []

    for ticket in other_tickets:
        valid_ticket = True
        for field in ticket:
            valid_field = False

            for poss_source in intervals.values():
                deb1, fin1, deb2, fin2 = poss_source
                if (deb1 <= field and field <= fin1) or (deb2 <= field and field <= fin2):
                    valid_field = True

            if not valid_field:
                valid_ticket = False
                total_invalid += field

        if valid_ticket:
            valid_tickets.append(ticket)

    # For each field, find all the positions that it could go (see if there's a lot)
    valid_indices = {field: [] for field in intervals}
    for field_name, bounds in intervals.items():
        deb1, fin1, deb2, fin2 = bounds
        #print(f'{field_name}: {bounds}') 
        for possible_index in range(len(my_ticket)):
            index_works = True
            for ticket in valid_tickets:
                val = ticket[possible_index]
                if not ((deb1 <= val and val <= fin1) or (deb2 <= val and val <= fin2)):
                    index_works = False
                    #print(f'  ticket {ticket} makes index {possible_index} fail:')
                    #print(f'    {val} is not in [{deb1}, {fin1}] or [{deb2}, {fin2}]')
                    break
            if index_works:
                valid_indices[field_name].append(possible_index)
        #print(f'{field_name} valid indices: {valid_indices[field_name]}\n')

    #print(valid_indices)

    found = {field: False for field in intervals}
    while sum(found.values()) < len(intervals):
        for field in intervals:
            if (len(valid_indices[field]) == 1) and (not found[field]):
                found[field] = True
                field_pos = valid_indices[field][0]
                for other_field in intervals:
                    if other_field != field and field_pos in valid_indices[other_field]:
                        valid_indices[other_field].remove(field_pos)
                #print(valid_indices)
    
    print(valid_indices)

    return valid_indices, my_ticket

In [143]:
nb_invalid(*prep_input(test_input_2))

{'class': [1], 'row': [0], 'seat': [2]}


({'class': [1], 'row': [0], 'seat': [2]}, [11, 12, 13])

In [124]:
with open('input_day16.txt', 'r') as data_file:
    full_input = data_file.read().split('\n\n')

In [144]:
final_rows, my_ticket = nb_invalid(*prep_input(full_input))

{'departure location': [5], 'departure station': [2], 'departure platform': [15], 'departure track': [1], 'departure date': [19], 'departure time': [11], 'arrival location': [13], 'arrival station': [17], 'arrival platform': [3], 'arrival track': [8], 'class': [6], 'duration': [0], 'price': [10], 'route': [9], 'row': [12], 'seat': [18], 'train': [7], 'type': [16], 'wagon': [14], 'zone': [4]}


In [153]:
def final_result(rows, ticket):
    product = 1
    print(ticket)
    for field, row in rows.items():
        if field.startswith('departure'):
            product *= ticket[row[0]]
    return product

In [154]:
final_result(*nb_invalid(*prep_input(full_input)))

{'departure location': [5], 'departure station': [2], 'departure platform': [15], 'departure track': [1], 'departure date': [19], 'departure time': [11], 'arrival location': [13], 'arrival station': [17], 'arrival platform': [3], 'arrival track': [8], 'class': [6], 'duration': [0], 'price': [10], 'route': [9], 'row': [12], 'seat': [18], 'train': [7], 'type': [16], 'wagon': [14], 'zone': [4]}
[191, 89, 73, 139, 71, 103, 109, 53, 97, 179, 59, 67, 79, 101, 113, 157, 61, 107, 181, 137]


964373157673

In [130]:
final_rows

{'departure location': [5],
 'departure station': [2],
 'departure platform': [15],
 'departure track': [1],
 'departure date': [19],
 'departure time': [11],
 'arrival location': [13],
 'arrival station': [17],
 'arrival platform': [3],
 'arrival track': [8],
 'class': [6],
 'duration': [0],
 'price': [10],
 'route': [9],
 'row': [12],
 'seat': [18],
 'train': [7],
 'type': [16],
 'wagon': [14],
 'zone': [4]}